# Simulation Boxes

A `Box` defines the simulation cell: its shape, periodic boundary conditions (PBC), and the coordinate transforms you need to work safely with periodic systems.

You will mostly use `Box` for:
- Wrapping coordinates into the primary cell (`wrap`)
- Minimum-image distance vectors and distances (`diff`, `dist`)
- Converting between absolute and fractional coordinates (`make_fractional`, `make_absolute`)

MolPy represents the box as a $3\times 3$ matrix whose **columns** are lattice vectors. From that matrix, MolPy can work with three common styles:
- **FREE**: no boundaries / no periodicity
- **ORTHOGONAL**: rectangular (diagonal matrix)
- **TRICLINIC**: general cell with tilts (off-diagonal terms)

## 1. Create boxes

Use factory constructors for common cells, or pass a matrix when you already have lattice vectors. Keep your unit system consistent with the coordinates you will store (Å, nm, etc.).

In [1]:
import molpy as mp
import numpy as np

cubic = mp.Box.cubic(20.0)
ortho = mp.Box.orth([10.0, 20.0, 30.0])

# LAMMPS-style triclinic: lengths + tilts (xy, xz, yz)
tric = mp.Box.tric(lengths=[10.0, 12.0, 15.0], tilts=[1.0, 0.5, 0.2])

# Direct matrix construction (columns are lattice vectors)
matrix = np.array([[10.0, 1.0, 0.5], [0.0, 12.0, 0.2], [0.0, 0.0, 15.0]])
box_from_matrix = mp.Box(matrix=matrix)

{
    "cubic_style": str(cubic.style),
    "ortho_lengths": ortho.lengths.tolist(),
    "tric_tilts": tric.tilts.tolist(),
    "matrix": box_from_matrix.matrix,
}

{'cubic_style': 'Style.ORTHOGONAL',
 'ortho_lengths': [10.0, 20.0, 30.0],
 'tric_tilts': [1.0, 0.5, 0.2],
 'matrix': array([[10. ,  1. ,  0.5],
        [ 0. , 12. ,  0.2],
        [ 0. ,  0. , 15. ]])}

## 2. Periodic boundary conditions (PBC)

PBC controls whether coordinates wrap around the box on each axis. Common setups:
- bulk system: periodic in `x/y/z`
- slab/interface: periodic in `x/y`, non-periodic in `z`

In [2]:
import molpy as mp

bulk = mp.Box.cubic(10.0)
slab = mp.Box.orth([20.0, 20.0, 50.0], pbc=[True, True, False])

{
    "bulk_pbc": bulk.pbc.tolist(),
    "slab_pbc": slab.pbc.tolist(),
    "slab_periodic_z": slab.periodic_z,
}

{'bulk_pbc': [True, True, True],
 'slab_pbc': [True, True, False],
 'slab_periodic_z': np.False_}

## 3. Geometry & derived properties

`Box` exposes derived properties from its matrix representation. You’ll typically inspect:
- `style`: FREE / ORTHOGONAL / TRICLINIC
- `lengths`, `volume`
- `origin`, `bounds`
- (triclinic only) `tilts`, `angles`

In [3]:
import molpy as mp
import numpy as np

ortho = mp.Box.orth([10.0, 12.0, 15.0])
tric = mp.Box.tric(lengths=[10.0, 12.0, 15.0], tilts=[1.0, 0.5, 0.2])

{
    "ortho": {
        "style": str(ortho.style),
        "lengths": ortho.lengths.tolist(),
        "volume": float(ortho.volume),
        "origin": ortho.origin.tolist(),
        "bounds": ortho.bounds.tolist(),
        "matrix": ortho.matrix,
    },
    "tric": {
        "style": str(tric.style),
        "lengths": tric.lengths.tolist(),
        "tilts": tric.tilts.tolist(),
        "angles_deg": tric.angles.tolist(),
        "volume": float(tric.volume),
        "matrix": tric.matrix,
    },
}

{'ortho': {'style': 'Style.ORTHOGONAL',
  'lengths': [10.0, 12.0, 15.0],
  'volume': 1800.000000000002,
  'origin': [0.0, 0.0, 0.0],
  'bounds': [[0.0, 0.0, 0.0], [10.0, 12.0, 15.0]],
  'matrix': array([[10.,  0.,  0.],
         [ 0., 12.,  0.],
         [ 0.,  0., 15.]])},
 'tric': {'style': 'Style.TRICLINIC',
  'lengths': [10.0, 12.041594578792296, 15.00966355385756],
  'tilts': [1.0, 0.5, 0.2],
  'angles_deg': [89.0806427441657, 88.09101712126736, 85.23635830927383],
  'volume': 1800.000000000002,
  'matrix': array([[10. ,  1. ,  0.5],
         [ 0. , 12. ,  0.2],
         [ 0. ,  0. , 15. ]])}}

## 4. Coordinate transforms & wrapping

Two safe patterns when working with PBC:
- Convert to fractional coordinates (easy to reason about periodic images)
- Wrap positions into the primary cell before analysis/export

In [4]:
import numpy as np
import molpy as mp

box = mp.Box.cubic(10.0)

points = np.array([
    [12.0, -2.0, 5.0],
    [25.0, 8.0, -3.0],
], dtype=float)

wrapped = box.wrap(points)
images = box.get_images(points)
unwrapped = box.unwrap(wrapped, images)

absolute = np.array([[5.0, 3.0, 7.0]], dtype=float)
fractional = box.make_fractional(absolute)
absolute_restored = box.make_absolute(fractional)

{
    "points": points,
    "wrapped": wrapped,
    "images": images,
    "unwrapped_matches_original": bool(np.allclose(unwrapped, points)),
    "fractional": fractional,
    "absolute_restored_matches": bool(np.allclose(absolute_restored, absolute)),
}

{'points': array([[12., -2.,  5.],
        [25.,  8., -3.]]),
 'wrapped': array([[2., 8., 5.],
        [5., 8., 7.]]),
 'images': array([[ 1, -1,  0],
        [ 2,  0, -1]]),
 'unwrapped_matches_original': True,
 'fractional': array([[0.5, 0.3, 0.7]]),
 'absolute_restored_matches': True}

## 5. Minimum-image distances

In a periodic system, the “closest” separation uses the minimum-image convention. Use:
- `diff(r1, r2)` for the minimum-image displacement vector
- `dist(r1, r2)` for the scalar distance

In [5]:
import numpy as np
import molpy as mp

box = mp.Box.cubic(10.0)

r1 = np.array([[1.0, 1.0, 1.0]], dtype=float)
r2 = np.array([[9.5, 9.5, 9.5]], dtype=float)

dr = box.diff(r1, r2)
d = box.dist(r1, r2)

points1 = np.array([[1.0, 1.0, 1.0], [2.0, 2.0, 2.0]], dtype=float)
points2 = np.array([[9.5, 9.5, 9.5], [8.0, 8.0, 8.0]], dtype=float)
distances = box.dist_all(points1, points2)

{
    "dr": dr,
    "distance": float(d),
    "dist_all_shape": distances.shape,
    "dist_all": distances,
}

/tmp/ipykernel_37829/103694416.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "distance": float(d),


{'dr': array([[1.5, 1.5, 1.5]]),
 'distance': 2.5980762113533142,
 'dist_all_shape': (2, 2),
 'dist_all': array([[2.59807621, 5.19615242],
        [4.33012702, 6.92820323]])}

## 6. Mini workflow: deterministic points + wrapping

A small, reproducible workflow you’ll do often:
1) define a box
2) create coordinates (here: a small grid)
3) shift them (some will fall outside)
4) wrap into the primary cell
5) compute a simple PBC-aware metric

In [6]:
import numpy as np
import molpy as mp

box = mp.Box.cubic(30.0, pbc=[True, True, True])

# Deterministic 5x5 grid in the xy plane (z=0), placed inside the box
grid = np.linspace(2.0, 28.0, 5)
xy = np.array([(x, y) for x in grid for y in grid], dtype=float)
points = np.column_stack([xy, np.zeros(len(xy))])

# Shift all points by +10 in x (some points now exceed 30.0)
shifted = points + np.array([10.0, 0.0, 0.0])
wrapped = box.wrap(shifted)

# PBC-aware pairwise distances on wrapped points
dist = box.dist_all(wrapped, wrapped)
np.fill_diagonal(dist, np.inf)
min_dist = float(dist.min())

{
    "box_lengths": box.lengths.tolist(),
    "n_points": int(wrapped.shape[0]),
    "any_outside_before_wrap": bool((shifted[:, 0] > box.lengths[0]).any()),
    "all_inside_after_wrap": bool(box.isin(wrapped).all()),
    "min_pair_distance": min_dist,
}

{'box_lengths': [30.0, 30.0, 30.0],
 'n_points': 25,
 'any_outside_before_wrap': True,
 'all_inside_after_wrap': True,
 'min_pair_distance': 3.999999999999999}

## Summary

- A `Box` is the cell geometry + PBC definition used for safe coordinate work.
- Construct common boxes with `cubic` / `orth` / `tric`, or provide a lattice matrix.
- Use `wrap` + fractional coordinates to keep periodic systems consistent.
- Use `diff` / `dist` / `dist_all` for minimum-image distances.